In [1]:
pip install google-cloud-vision

In [26]:
pip install opencv-python

In [316]:
pip install google-cloud-automl

In [10]:
import pandas as pd
import numpy as np

In [8]:
# Importar excel
import pandas as pd

base = pd.read_excel('C:\\Users\\ext_fantmiraudal\\Desktop\\base_imagenes_etiquetadas.xlsx')

In [11]:
# Mostrar base
print('Tamaño:', base.shape) 
base.head()

Tamaño: (17139, 33)


,Suborden,Imageurl,Producto,Verde,Naranja,Negro,Caja Falabella,Bolsa,Caja,Cantidad de Bultos,...,Guia (Despacho/Retiro),Foto B/W,Suelo,Imagen truncada,Interiores de Movil,Carretera,Cielo,Fachada,ERROR URL,Columna1
0,149046188492,https://firebasestorage.googleapis.com/v0/b/tx...,1,0,1,0,0,0,0,1,...,0,0,0,0,0,0,0,0,0,NaN
1,149046194567,https://firebasestorage.googleapis.com/v0/b/tx...,1,1,0,0,0,0,0,1,...,0,0,0,0,0,0,0,0,0,NaN
2,149046200287,https://firebasestorage.googleapis.com/v0/b/tx...,1,0,0,0,1,0,0,1,...,0,0,0,0,0,0,0,0,0,NaN
3,149046194290,https://firebasestorage.googleapis.com/v0/b/tx...,1,1,0,0,0,0,0,1,...,0,0,0,0,0,0,0,0,0,NaN
4,149046201605,https://firebasestorage.googleapis.com/v0/b/tx...,1,0,1,0,0,0,0,1,...,0,0,0,0,0,0,0,0,0,NaN


In [188]:
base.iloc[0]['Imageurl']

'https://firebasestorage.googleapis.com/v0/b/txd-scex-tmgt-geosort-prod.appspot.com/o/delivery%2Fvsantis%40pipau.cl%2F149046188492.png?alt=media&token=27a2ccda-45d7-4da5-9f43-14db38bc77d7'

In [18]:
# Aplicar filtros (Imageurl, Producto, Rostro cliente)
base_prueba = base[['Suborden','Imageurl', 'Producto', 'Rostro cliente']]
base_prueba.head()

,Suborden,Imageurl,Producto,Rostro cliente
0,149046188492,https://firebasestorage.googleapis.com/v0/b/tx...,1,0
1,149046194567,https://firebasestorage.googleapis.com/v0/b/tx...,1,0
2,149046200287,https://firebasestorage.googleapis.com/v0/b/tx...,1,0
3,149046194290,https://firebasestorage.googleapis.com/v0/b/tx...,1,0
4,149046201605,https://firebasestorage.googleapis.com/v0/b/tx...,1,0


In [149]:
# Extraer subconjunto aleatorio 
base_prueba1 = base_prueba.sample(n = 1000, random_state = 2, replace = False)

In [150]:
base_prueba1

,Suborden,Imageurl,Producto,Rostro cliente
9877,149046204462,https://firebasestorage.googleapis.com/v0/b/tx...,1,1
3982,149045389414,https://firebasestorage.googleapis.com/v0/b/tx...,1,0
16534,149048022673,https://firebasestorage.googleapis.com/v0/b/tx...,1,0
14106,149048586672,https://firebasestorage.googleapis.com/v0/b/tx...,0,0
16647,149048571125,https://firebasestorage.googleapis.com/v0/b/tx...,0,0
...,...,...,...,...
10324,149046063287,https://firebasestorage.googleapis.com/v0/b/tx...,1,0
17059,149048343732,https://firebasestorage.googleapis.com/v0/b/tx...,1,0
13871,149048850675,https://firebasestorage.googleapis.com/v0/b/tx...,0,0
16845,149048362269,https://firebasestorage.googleapis.com/v0/b/tx...,1,1


In [216]:
# Aplicar vision api solo a una imagen
def vision_api(url, jason_path, t_product = 0.5, t_face = 0.5):
    import os
    from google.cloud import vision
    import io
    import cv2
    import numpy as np
    import urllib
    import pandas as pd
    
    if t_product < 0.5 or t_face < 0.5:
        print('Umbral de confianza no puede ser menor a 0.5')
        return
    
    # Credenciales
    os.environ['GOOGLE_APPLICATION_CREDENTIALS'] = jason_path
    # Instanciar cliente
    client = vision.ImageAnnotatorClient()
    
    # Vector de salida (fila) prob. producto, producto, prob. sin rostro cliente, sin rostro cliente, contexto
    tamaño = (1,5)
    result = np.zeros(tamaño)
    
    # Se lee imagen
    image = vision.Image()
    image.source.image_uri = url
    
    # Obtener área total de la imagen
    url_open = urllib.request.urlopen(url)
    image_cv = np.asarray(bytearray(url_open.read()), dtype="uint8")
    image_cv = cv2.imdecode(image_cv, cv2.IMREAD_COLOR)
    area_total = image_cv.shape[0]*image_cv.shape[1]
    
    # Detección de objetos
    objects = client.object_localization(image=image).localized_object_annotations
    while client.object_localization(image=image).error.code == 14:
        print('Fallo en lectura de URL. Intentando nuevamente.\n')
    # Ver si en objetos hay algun producto
    # Contador de productos
    contador = 0
    # Inicializar area de producto
    area_producto = 0
    # Inicializar lista de confianzas
    confidence_list = []
    for object_ in objects:
        if (object_.name == 'Luggage & bags') or (object_.name == 'Bagged packaged goods') \
            or (object_.name =='Packaged goods') or (object_.name =='Boxed packaged goods') \
            or (object_.name =='Bag') or (object_.name =='Shipping box'):
            # Guardar confianza de la detección
            confidence_list.append(object_.score)
            # Guardar coordenadas
            coordenadas = []
            for vertex in object_.bounding_poly.normalized_vertices:
                coordenadas.append((vertex.x, vertex.y))          
            # Punto superior izquierdo del rectángulo
            x0 = (int(coordenadas[0][0]*image_cv.shape[1]), int(coordenadas[0][1]*image_cv.shape[0]))
            # Punto inferior derecho del rectángulo
            x1 = (int(coordenadas[2][0]*image_cv.shape[1]), int(coordenadas[2][1]*image_cv.shape[0])) 
            # Sumar áreas de cada producto
            area_producto += abs((x0[0]-x1[0])*(x0[1]-x1[1]))
            contador+=1
     
    if contador != 0:
        # Rellenar con la confianza máxima la posición correspondiente en la fila
        result[0][0] += max(confidence_list)
        
        if max(confidence_list) >= t_product:
            # Rellenar con 1 la posición correspondiente en la fila
            result[0][1] += 1       
         
        # Calcular contexto
        # Dividir por área total
        razon_areas = area_producto/area_total
        # Rellenar contexto en fila
        result[0][4] = razon_areas
    
    # Detección de caras
    response = client.face_detection(image=image)
    faces = response.face_annotations
    print(client.face_detection(image=image).error)
    
    # Inicializar lista de confianzas
    confidence_list = []
    for face in faces:
        # Guardar confianza de la detección
        confidence_list.append(face.detection_confidence)
        
    if len(faces) != 0: # Si ocurrre esto es por que hay detección
        # Rellenar con 1 menos la confianza máxima (confianza de que no haya un rostro) 
        # la posición correspondiente en la fila
        result[0][2] += (1-max(confidence_list))
        
        if (1-max(confidence_list)) > 1 - t_face:
            # Rellenar con 1 la posición correspondiente en la matriz
            result[0][3] += 1
    
    result_data = pd.DataFrame(result, columns=['Prob. Producto', 'Producto', 'Prob. Sin rostro cliente', 'Sin rostro cliente',\
                                                'Contexto'])
    return result_data    

In [221]:
vision_api('https://firebasestorage.googleapis.com/v0/b/txd-scex-tmgt-geosort-prod.appspot.com/o/delivery%2Fepalomera%40pipau.cl%2F149043759299.png?alt=media&token=f61d27c2-9072-418b-86b2-82ed032d7a1a', "clave2\\tc-sc-bi-bigdata-corp-tsod-dev-739359c1cf2a.json")

,Prob. Producto,Producto,Prob. Sin rostro cliente,Sin rostro cliente,Contexto
0,0.807049,1.0,0.0,0.0,0.696627


In [37]:
# Aplicar vision api solo a una imagen
def vision_api(url, jason_path, t_product = 0.5, t_face = 0.5):
    #ver si se puede almacenar imagen dentro del codigo y borrarlo al terminar ejecución
    import os
    from google.cloud import vision
    import io
    import cv2
    import numpy as np
    import urllib
    import pandas as pd
    
    # Credenciales
    os.environ['GOOGLE_APPLICATION_CREDENTIALS'] = jason_path
    # Instanciar cliente
    client = vision.ImageAnnotatorClient()
    
    # Vector de salida (fila) prob. producto, producto, prob. sin rostro cliente, sin rostro cliente, contexto
    tamaño = (1,3)
    result = np.zeros(tamaño)
    
    # Se lee imagen
    image = vision.Image()
    image.source.image_uri = url
    
    # Obtener área total de la imagen
    url_open = urllib.request.urlopen(url)
    image_cv = np.asarray(bytearray(url_open.read()), dtype="uint8")
    image_cv = cv2.imdecode(image_cv, cv2.IMREAD_COLOR)
    area_total = image_cv.shape[0]*image_cv.shape[1]
    #print(area_total)
    
    #with open(path, 'rb') as image_file:
    #    content = image_file.read()
    #image = vision.Image(content=content)
    b = bytes(image_cv)
    #print(b)
    #print(image)
    image2 = vision.Image(content=b)
    response = client.face_detection(image=image2)
    faces2 = response.face_annotations
    #print(faces2)
    objects2 = client.object_localization(image=image2).localized_object_annotations
    print(client.object_localization(image=image2))
    print(client.object_localization(image=image2).error.message)
    
    # Detección de objetos
    objects = client.object_localization(image=image).localized_object_annotations
    print(client.object_localization(image=image))
    from pprint import pprint
    pprint(dir(client.object_localization(image=image)))
    print('imprimir error: \n')
    print(client.object_localization(image=image).error.message)
    print(client.object_localization(image=image).__contains__)
    
    i=2
    print('imprimir if: \n')
    if client.object_localization(image=image).error.message:
        print(i)
    # Ver si en objetos hay algun producto
    # Contador de productos
    contador = 0
    # Inicializar area de producto
    area_producto = 0
    # Inicializar lista de confianzas
    confidence_list = []
    for object_ in objects:
        print(object_)
        if (object_.name == 'Luggage & bags') or (object_.name == 'Bagged packaged goods') \
            or (object_.name =='Packaged goods') or (object_.name =='Boxed packaged goods') \
            or (object_.name =='Bag') or (object_.name =='Shipping box'):
            # Guardar confianza de la detección
            confidence_list.append(object_.score)
            # Guardar coordenadas
            coordenadas = []
            for vertex in object_.bounding_poly.normalized_vertices:
                coordenadas.append((vertex.x, vertex.y))          
            # Punto superior izquierdo del rectángulo
            x0 = (int(coordenadas[0][0]*image_cv.shape[1]), int(coordenadas[0][1]*image_cv.shape[0]))
            # Punto inferior derecho del rectángulo
            x1 = (int(coordenadas[2][0]*image_cv.shape[1]), int(coordenadas[2][1]*image_cv.shape[0])) 
            # Sumar áreas de cada producto
            area_producto += abs((x0[0]-x1[0])*(x0[1]-x1[1]))
            contador+=1
     
    if contador != 0:
        # Rellenar con 1 la posición correspondiente en la fila
        #result[0][0] +=1
        # Rellenar con la confianza máxima la posición correspondiente en la fila
        result[0][0] += max(confidence_list)
        
        #if max(confidence_list) >= t_product:
            
         
        # Calcular contexto
        # Dividir por área total
        razon_areas = area_producto/area_total
        # Rellenar contexto en fila
        result[0][2] = razon_areas
    
    # Detección de caras
    response = client.face_detection(image=image)
    faces = response.face_annotations
    print('wololoooo:\n')
    print(client.face_detection(image=image2).error)
    
    # Inicializar lista de confianzas
    confidence_list = []
    for face in faces:
        # Guardar confianza de la detección
        confidence_list.append(face.detection_confidence)
        
    if len(faces) != 0: # Si ocurrre esto es por que hay detección
        # Rellenar con 1 la posición correspondiente en la matriz
        #result[0][1] +=1
        # Rellenar con 1 menos la confianza máxima (confianza de que no haya un rostro) 
        # la posición correspondiente en la fila
        result[0][1] += (1-max(confidence_list))
    
    result_data = pd.DataFrame(result, columns=['Producto','Sin rostro cliente', 'Contexto'])
    return result_data    

In [38]:
vision_api('https://firebasestorage.googleapis.com/v0/b/txd-scex-tmgt-geosort-prod.appspot.com/o/delivery%2Flileonb%40intralogistics.cl%2F149045819215.png?alt=media&token=03d6c2db-e887-4140-8dc0-0308f8ee7d36', "clave2\\tc-sc-bi-bigdata-corp-tsod-dev-739359c1cf2a.json")

error {
  code: 3
  message: "Bad image data."
}

Bad image data.
localized_object_annotations {
  mid: "/m/07mhn"
  name: "Pants"
  score: 0.9499415159225464
  bounding_poly {
    normalized_vertices {
      x: 0.3957786560058594
      y: 0.5798817276954651
    }
    normalized_vertices {
      x: 0.6585014462471008
      y: 0.5798817276954651
    }
    normalized_vertices {
      x: 0.6585014462471008
      y: 0.9093387126922607
    }
    normalized_vertices {
      x: 0.3957786560058594
      y: 0.9093387126922607
    }
  }
}
localized_object_annotations {
  mid: "/m/06rrc"
  name: "Shoe"
  score: 0.9132254719734192
  bounding_poly {
    normalized_vertices {
      x: 0.5622024536132812
      y: 0.8640679717063904
    }
    normalized_vertices {
      x: 0.6782727241516113
      y: 0.8640679717063904
    }
    normalized_vertices {
      x: 0.6782727241516113
      y: 0.9732323288917542
    }
    normalized_vertices {
      x: 0.5622024536132812
      y: 0.9732323288917542
    }
  }

mid: "/m/07mhn"
name: "Pants"
score: 0.9499415159225464
bounding_poly {
  normalized_vertices {
    x: 0.3957786560058594
    y: 0.5798817276954651
  }
  normalized_vertices {
    x: 0.6585014462471008
    y: 0.5798817276954651
  }
  normalized_vertices {
    x: 0.6585014462471008
    y: 0.9093387126922607
  }
  normalized_vertices {
    x: 0.3957786560058594
    y: 0.9093387126922607
  }
}

mid: "/m/06rrc"
name: "Shoe"
score: 0.9132254719734192
bounding_poly {
  normalized_vertices {
    x: 0.5622024536132812
    y: 0.8640679717063904
  }
  normalized_vertices {
    x: 0.6782727241516113
    y: 0.8640679717063904
  }
  normalized_vertices {
    x: 0.6782727241516113
    y: 0.9732323288917542
  }
  normalized_vertices {
    x: 0.5622024536132812
    y: 0.9732323288917542
  }
}

mid: "/m/01g317"
name: "Person"
score: 0.8864565491676331
bounding_poly {
  normalized_vertices {
    x: 0.3089944124221802
    y: 0.14072392880916595
  }
  normalized_vertices {
    x: 0.7594544291496277
    y:

,Producto,Sin rostro cliente,Contexto
0,0.577439,0.19921,0.085907


In [60]:
project_id = "tc-sc-bi-bigdata-corp-tsod-dev"
model_id = "ICN2033373032734523392"
file_path = "test\\numero_domicilio\\internet0.jpg"
predict(project_id, model_id, file_path)

PermissionDenied: 403 The caller does not have permission

In [54]:
from google.cloud import automl
import os

# TODO(developer): Uncomment and set the following variables
project_id = "tc-sc-bi-bigdata-corp-tsod-dev"

jason_path = "key\\tc-sc-bi-bigdata-corp-tsod-dev-82c15beedd2a.json"
#jason_path = 'tc-sc-bi-bigdata-corp-tsod-dev-de643e35c621.json'
os.environ['GOOGLE_APPLICATION_CREDENTIALS'] = jason_path

client = automl.AutoMlClient()
# A resource that represents Google Cloud Platform location.
project_location = f"projects/{project_id}/locations/us-central1"

request = automl.ListModelsRequest(parent=project_location, filter="")
response = client.list_models(request=request)

print("List of models:")
for model in response:
    # Display the model information.
    if model.deployment_state == automl.Model.DeploymentState.DEPLOYED:
        deployment_state = "deployed"
    else:
        deployment_state = "undeployed"

    print("Model name: {}".format(model.name))
    print("Model id: {}".format(model.name.split("/")[-1]))
    print("Model display name: {}".format(model.display_name))
    print("Model create time: {}".format(model.create_time))
    print("Model deployment state: {}".format(deployment_state))

PermissionDenied: 403 The caller does not have permission

In [5]:
# Aplicar AutoMl solo a una imagen
def auto_ml(url, jason_path):
    from google.cloud import automl
    import os
    import urllib
    import numpy as np
    #from numpy.ndarray import tobytes
    ############################################################jason_path = "key\\tc-sc-bi-bigdata-corp-tsod-dev-82c15beedd2a.json"
    os.environ['GOOGLE_APPLICATION_CREDENTIALS'] = jason_path
    
    # TODO(developer): Uncomment and set the following variables
    project_id = "tc-sc-bi-bigdata-corp-tsod-dev"
    #model_id_1 = "ICN2033373032734523392"
    model_id ="ICN5571024207313108992"
    #file_path = "test\\numero_domicilio\\internet0.jpg"
    
    prediction_client = automl.PredictionServiceClient()
    
    # Get the full path of the model.
    model_full_id = automl.AutoMlClient.model_path(project_id, "us-central1", model_id)
    #print(model_full_id)
    
    # Read the file.
    #with open(file_path, "rb") as content_file:
    #    content = content_file.read()
    #    print(content)
        
    # Obtener área total de la imagen
    ############################################################url = 'https://firebasestorage.googleapis.com/v0/b/txd-scex-tmgt-geosort-prod.appspot.com/o/delivery%2Flmriquelme%40corsario.cl%2F149046100374.png?alt=media&token=990a6582-4e44-4a85-8b66-68386edbcedf'
    url_open = urllib.request.urlopen(url)
    content = url_open.read()
    
    image = automl.Image(image_bytes=content)
    payload = automl.ExamplePayload(image=image)
    
    # params is additional domain-specific parameters.
    # score_threshold is used to filter the result
    # https://cloud.google.com/automl/docs/reference/rpc/google.cloud.automl.v1#predictrequest
    params = {"score_threshold": "0.5"}
    
    request = automl.PredictRequest(name=model_full_id, payload=payload, params=params)
    response = prediction_client.predict(request=request)
    
    # Vector de salida (fila) etiqueta_producto, numero_domicilio
    tamaño = (1,2)
    result = np.zeros(tamaño)
    
    for resultado in response.payload:
        if resultado.display_name == 'etiqueta_producto':
            result[0,0] = resultado.classification.score
        elif resultado.display_name == 'numero_domicilio':
            result[0,1] = resultado.classification.score
        
    result_data = pd.DataFrame(result, columns=['Etiqueta producto','Numero domicilio'])
    return result_data    
    
    #print(response.payload)
    #print("Prediction results:")
    #for result in response.payload:
    #    print("Predicted class name: {}".format(result.display_name))
    #    print("Predicted class score: {}".format(result.classification.score))
    

In [4]:
# Función para calcular score
def calculo_score(data_frame):
    import math
    array = np.array(data_frame)
    w_product = 0.3
    w_without_face = 0.1
    w_context = 0.2
    w_product_label = 0.3
    w_address_number = 0.1
    k = 55.3
    new_context = math.exp(-(data_frame['Contexto'][0]-0.5)**2*k)
    score = data_frame['Producto'][0]*w_product + data_frame['Sin rostro cliente'][0]*w_without_face + \
    new_context*w_context + data_frame['Etiqueta producto'][0]*w_product_label + \
    data_frame['Numero domicilio'][0]*w_address_number
    
    return score

In [2]:
# Función que retorna las confidencias + el score en un dataframe
def full_model(url,jason_path_api, jason_path_automl):
    import pandas as pd
    df_1 = vision_api(url, jason_path_api)
    df_2 = auto_ml(url, jason_path_automl)
    df_concatenado = pd.concat([df_1, df_2], axis=1)
    score = calculo_score(df_concatenado)
    df_score = pd.DataFrame(np.array([score]), columns=['Score'])
    return pd.concat([df_concatenado, df_score], axis=1)

In [14]:
import pandas as pd
import numpy as np
url="https://firebasestorage.googleapis.com/v0/b/txd-scex-tmgt-geosort-prod.appspot.com/o/delivery%2Fjavillalobos%40pipau.cl%2F149045394625.png?alt=media&token=335839a5-d0f5-4e5a-bff7-206fac166528"
jason_path_api= "clave2\\tc-sc-bi-bigdata-corp-tsod-dev-739359c1cf2a.json"
jason_path_automl="key\\tc-sc-bi-bigdata-corp-tsod-dev-82c15beedd2a.json"
jason_path_api= "key\\tc-sc-bi-bigdata-corp-tsod-dev-82c15beedd2a.json"
full_model(url,jason_path_api, jason_path_automl)

,Producto,Sin rostro cliente,Contexto,Etiqueta producto,Numero domicilio,Score
0,0.892741,0.172745,0.401797,0.870319,0.0,0.663526


In [29]:
data1 = pd.DataFrame(np.array([2]), columns=['Producto'])
data2 = pd.DataFrame(np.zeros((1,2)), columns=['etiqueta_producto','numero_domicilio'])
data_concatenado = pd.concat([data1, data2], axis=1)
data_concatenado

,Producto,etiqueta_producto,numero_domicilio
0,2,0.0,0.0


In [102]:
data_concatenado['Producto'][0]

0.0

In [90]:
np.array(data_concatenado)

array([[0., 0., 0., 0., 0.]])

In [92]:
array = np.array(data_concatenado)
for i in array[0]:
    

0.0
0.0
0.0
0.0
0.0


In [41]:
from google.cloud import automl
import os
jason_path ="auto_ml_jason\\tc-sc-bi-bigdata-corp-tsod-dev-05c522e8964b.json"
os.environ["GOOGLE_APPLICATION_CREDENTIALS"]=jason_path

# TODO(developer): Uncomment and set the following variables
project_id = "tc-sc-bi-bigdata-corp-tsod-dev"
model_id = "ICN2033373032734523392"
file_path = "test\\numero_domicilio\\internet0.jpg"

prediction_client = automl.PredictionServiceClient()

# Get the full path of the model.
model_full_id = automl.AutoMlClient.model_path(project_id, "us-central1", model_id)

# Read the file.
with open(file_path, "rb") as content_file:
    content = content_file.read()

image = automl.Image(image_bytes=content)
payload = automl.ExamplePayload(image=image)

# params is additional domain-specific parameters.
# score_threshold is used to filter the result
# https://cloud.google.com/automl/docs/reference/rpc/google.cloud.automl.v1#predictrequest
params = {"score_threshold": "0.8"}

request = automl.PredictRequest(name=model_full_id, payload=payload, params=params)
response = prediction_client.predict(request=request)

print("Prediction results:")
for result in response.payload:
    print("Predicted class name: {}".format(result.display_name))
    print("Predicted class score: {}".format(result.classification.score))

PermissionDenied: 403 The caller does not have permission

In [55]:
import sys

from google.cloud import automl_v1beta1
from google.cloud.automl_v1beta1.proto import service_pb2


# 'content' is base-64-encoded image data.
def get_prediction(content, project_id, model_id):
    prediction_client = automl_v1beta1.PredictionServiceClient()
    name = 'projects/{}/locations/us-central1/models/{}'.format(project_id, model_id)
    payload = {'image': {'image_bytes': content }}
    params = {}
    request = prediction_client.predict(name, payload, params)
    return request  # waits till request is returned

if __name__ == '__main__':
    file_path = sys.argv[1]
    project_id = sys.argv[2]
    model_id = sys.argv[3]
    with open(file_path, 'rb') as ff:
        content = ff.read()
        
    print(get_prediction(content, project_id, model_id))

ModuleNotFoundError: No module named 'google.cloud.automl_v1beta1.proto'

In [151]:
# Aplicar vision api a un conjunto de datos
#ARREGLAAR OR!!!!!!
def vision_api_data(data, jason_path):
    import os
    from google.cloud import vision
    import io
    import cv2
    import numpy as np
    
    # Credenciales
    os.environ['GOOGLE_APPLICATION_CREDENTIALS'] = jason_path
    # Instanciar cliente
    client = vision.ImageAnnotatorClient()
    
    # Matriz de salida
    tamaño = data.shape
    result = np.zeros(tamaño)
    
    # Se instancia fila y columna 
    fila=0
    #Suborden = data.columns.get_loc('Suborden')
    columna_producto=data.columns.get_loc('Producto')
    columna_rostro=data.columns.get_loc('Rostro cliente')
    # Se recorren los enlaces
    for enlace in data['Imageurl']:
        image = vision.Image()
        image.source.image_uri = enlace
        
        # Detección de objetos
        objects = client.object_localization(image=image).localized_object_annotations
        
        #result[fila][Suborden] = data.iloc[fila]['Suborden']
        
        # Ver si en objetos hay algun producto
        for object_ in objects:
            if object_.name == ('Packaged goods' or 'Boxed packaged goods' or 'Bag' or 'Shipping box'):
                # Rellenar con 1 la posición correspondiente en la matriz
                result[fila][columna_producto] +=1
                break    
        
        # Detección de caras
        response = client.face_detection(image=image)
        faces = response.face_annotations
        
        if len(faces) != 0: # Si ocurrre esto es por que hay detección
            # Rellenar con 1 la posición correspondiente en la matriz
            result[fila][columna_rostro] +=1
        
        # Siguiente fila
        fila+=1
    
    result_data = pd.DataFrame(result, columns=['-','-','Producto','Rostro cliente'])
    return result_data    
    

In [152]:
result = vision_api_data(base_prueba1, 'tc-sc-bi-bigdata-corp-tsod-dev-de643e35c621.json')

In [153]:
# Calcular error c/r metodo manual
from sklearn.metrics import mean_squared_error
import numpy as np
#-> calcular accuracy mejor!
#-> para el contexto revisar una cierta cantidad de fotos para visualizar (niveles de contexto distinto)
#-> aproach sumar area o area máxima
mse_product = mean_squared_error(np.array(base_prueba1['Producto']), np.array(result['Producto']))
print('mse_product: ', mse_product)
mse_face = mean_squared_error(np.array(base_prueba1['Rostro cliente']), np.array(result['Rostro cliente']))
print('mse_face: ', mse_face)
total_error = (mse_product + mse_face)/2
print('total_error: ', total_error)

mse_product:  0.381
mse_face:  0.057
total_error:  0.219


In [155]:
total_error

0.219

In [177]:
etiqueta_producto = base[(base['Etiqueta Envio'] == 1) & (base['Numeracion del Domicilio'] == 0)]
etiqueta_producto

,Suborden,Imageurl,Producto,Verde,Naranja,Negro,Caja Falabella,Bolsa,Caja,Cantidad de Bultos,...,Guia (Despacho/Retiro),Foto B/W,Suelo,Imagen truncada,Interiores de Movil,Carretera,Cielo,Fachada,ERROR URL,Columna1
0,149046188492,https://firebasestorage.googleapis.com/v0/b/tx...,1,0,1,0,0,0,0,1,...,0,0,0,0,0,0,0,0,0,NaN
1,149046194567,https://firebasestorage.googleapis.com/v0/b/tx...,1,1,0,0,0,0,0,1,...,0,0,0,0,0,0,0,0,0,NaN
2,149046200287,https://firebasestorage.googleapis.com/v0/b/tx...,1,0,0,0,1,0,0,1,...,0,0,0,0,0,0,0,0,0,NaN
3,149046194290,https://firebasestorage.googleapis.com/v0/b/tx...,1,1,0,0,0,0,0,1,...,0,0,0,0,0,0,0,0,0,NaN
4,149046201605,https://firebasestorage.googleapis.com/v0/b/tx...,1,0,1,0,0,0,0,1,...,0,0,0,0,0,0,0,0,0,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
17131,149048716071,https://firebasestorage.googleapis.com/v0/b/tx...,1,0,0,0,0,0,1,2,...,0,0,1,0,0,0,0,0,0,FOTO SOLO PRODUCTO
17132,149048333507,https://firebasestorage.googleapis.com/v0/b/tx...,1,0,1,0,0,1,1,3,...,0,0,1,0,0,0,0,0,0,SE APRECIA PRODUCTO Y DOMICILIO/CLIENTE
17134,149048420584,https://firebasestorage.googleapis.com/v0/b/tx...,1,0,0,0,0,0,1,1,...,0,0,1,0,0,0,0,1,0,SE APRECIA PRODUCTO Y DOMICILIO/CLIENTE
17136,149048178115,https://firebasestorage.googleapis.com/v0/b/tx...,1,0,0,0,0,0,1,1,...,0,0,1,0,0,0,0,1,0,SE APRECIA PRODUCTO Y DOMICILIO/CLIENTE


In [180]:
# Descargar imagenes etiquetadas
# importing required modules
import urllib.request

# Filtros 
etiqueta_producto = base[(base['Etiqueta Envio'] == 1) & (base['Numeracion del Domicilio'] == 0)]
etiqueta_y_numero = base[(base['Etiqueta Envio'] == 1) & (base['Numeracion del Domicilio'] == 1)]
numero_domicilio = base[(base['Etiqueta Envio'] == 0) & (base['Numeracion del Domicilio'] == 1)]


i=0
for image_url in etiqueta_producto['Imageurl']:
    # setting filename and image URL
    filename = 'Data etiquetada//etiqueta_producto//sample_' + str(i) + '.png'
    
    # calling urlretrieve function to get resource
    urllib.request.urlretrieve(image_url, filename)
    i+=1

i=0
for image_url in etiqueta_y_numero['Imageurl']:
    # setting filename and image URL
    filename = 'Data etiquetada//etiqueta_y_numero//sample_' + str(i) + '.png'
    
    # calling urlretrieve function to get resource
    urllib.request.urlretrieve(image_url, filename)
    i+=1

i=0
for image_url in numero_domicilio['Imageurl']:
    # setting filename and image URL
    filename = 'Data etiquetada//numero_domicilio//sample_' + str(i) + '.png'
    
    # calling urlretrieve function to get resource
    urllib.request.urlretrieve(image_url, filename)
    i+=1

FileNotFoundError: [Errno 2] No such file or directory: 'Data etiquetada//ninguno//sample_0.png'

In [191]:
nada = base[(base['Etiqueta Envio'] == 0) & (base['Numeracion del Domicilio'] == 0)]

i=0
for image_url in nada['Imageurl']:
    if nada.iloc[i]['ERROR URL'] == 0:
        # setting filename and image URL
        filename = 'Data etiquetada//nada//sample_' + str(i) + '.png'
        
        # calling urlretrieve function to get resource
        urllib.request.urlretrieve(image_url, filename)
        i+=1
        if i == 6643:
            break
    else:
        i+=1

HTTPError: HTTP Error 403: Forbidden

In [63]:
def localize_objects_uri(uri, jason_path):
    """Localize objects in the image on Google Cloud Storage

    Args:
    uri: The path to the file in Google Cloud Storage (gs://...)
    """
    import os
    from google.cloud import vision
    import io
    import cv2
    import numpy as np
    
    # Credenciales
    os.environ['GOOGLE_APPLICATION_CREDENTIALS'] = jason_path
    
    # Instanciar cliente
    client = vision.ImageAnnotatorClient()

    image = vision.Image()
    image.source.image_uri = uri

    objects = client.object_localization(
        image=image).localized_object_annotations
    
    print(objects)
    'Pants' in objects 
    
    print('Number of objects found: {}'.format(len(objects)))
    for object_ in objects:
        print('\n{} (confidence: {})'.format(object_.name, object_.score))
        print('Normalized bounding polygon vertices: ')
        for vertex in object_.bounding_poly.normalized_vertices:
            print(' - ({}, {})'.format(vertex.x, vertex.y))

In [64]:
localize_objects_uri('https://firebasestorage.googleapis.com/v0/b/txd-scex-tmgt-geosort-prod.appspot.com/o/delivery%2Farhueica%40saetas.cl%2F149045316432.png?alt=media&token=e62599e5-5405-4f00-b89c-b3e685eae190', 'tc-sc-bi-bigdata-corp-tsod-dev-de643e35c621.json')

[]
Number of objects found: 0


In [254]:
# Detección imagen local
def detect_objects(image_path, jason_path):
    """Detects objects in an image."""
    import os
    from google.cloud import vision
    import io
    import cv2
    import numpy as np
    
    # Credenciales
    os.environ['GOOGLE_APPLICATION_CREDENTIALS'] = jason_path
    
    # Instanciar cliente
    client = vision.ImageAnnotatorClient()

    # Pasar imagen a 
    with io.open(image_path, 'rb') as image_file:
        content = image_file.read()

    image = vision.Image(content=content)
    
    image_cv = cv2.imread(image_path)
    print(image_cv.shape)
    window_name = 'Image'
    
    # Reconocimiento de objetos
    i=1
    objects = client.object_localization(image=image).localized_object_annotations
    print('Number of objects found: {}'.format(len(objects)))
    for object_ in objects:
        print('\n '+ 'Figura'+ str(i))
        print('object: {}'.format(object_.name, object_.score))
        print('confidence: {}'.format(object_.score))
        #print('Normalized bounding polygon vertices: ')
        coordenadas = []
        #print(object_)
        for vertex in object_.bounding_poly.normalized_vertices:
            coordenadas.append((vertex.x, vertex.y))
            #print(coordenadas)
            #print(' - ({}, {})'.format(vertex.x, vertex.y))
        
            
        # Dibujar borde
        #print(np.linalg.norm(coordenadas[0]))
        # represents the top left corner of rectangle
        start_point = (int(coordenadas[0][0]*image_cv.shape[1]), int(coordenadas[0][1]*image_cv.shape[0]))
        #print(start_point)

        # represents the bottom right corner of rectangle
        end_point = (int(coordenadas[2][0]*image_cv.shape[1]), int(coordenadas[2][1]*image_cv.shape[0]))
        #print(end_point)

        # Blue color in BGR
        color = (0, 255, 0)

        # Line thickness of 2 px
        thickness = 1
        
        image_cv = cv2.rectangle(image_cv, start_point, end_point, color, thickness)
        
        # Colocar texto
        
        #text = str(round((face.detection_confidence),2)) + 'figura ' + str(i)
        #text = 'figura ' + str(i)
        text = object_.name
        i+=1
        #print(coordenadas[0][0])
        #position = (coordenadas[0][0], coordenadas[0][1]+10)
        position = start_point
        font_family = cv2.FONT_HERSHEY_SIMPLEX
        font_size = 0.37
        font_color = (0, 255, 0)
        font_stroke = 1
        
        image_cv = cv2.putText(image_cv, text, position, font_family, font_size, font_color, font_stroke)
        cv2.imshow(window_name, image_cv)
        cv2.waitKey(0) 
        
    #cv2.imshow(window_name, image_cv)
    #cv2.waitKey(0)  

In [86]:
# Detección imagen local
def detect_objects(image_path, jason_path):
    """Detects objects in an image."""
    import os
    from google.cloud import vision
    import io
    import cv2
    import numpy as np
    
    # Credenciales
    os.environ['GOOGLE_APPLICATION_CREDENTIALS'] = jason_path
    
    # Instanciar cliente
    client = vision.ImageAnnotatorClient()

    # Codificación
    print(image_path)
    with io.open(image_path, 'rb') as image_file:
        content = image_file.read()

    image = vision.Image(content=content)
    
    image_cv = cv2.imread(image_path)
    window_name = 'Image'
    
    # Reconocimiento de objetos
    i=1
    objects = client.object_localization(image=image).localized_object_annotations
    
    if 'Bag' in objects:
        print('true')
    print('Number of objects found: {}'.format(len(objects)))
    for object_ in objects:
        print(object_.score)
        print('\n '+ 'Figura'+ str(i))
        print(object_.name)
        print('object: {}'.format(object_.name, object_.score))
        print('confidence: {}'.format(object_.score))
        
        # Guardar coordenadas
        coordenadas = []
        for vertex in object_.bounding_poly.normalized_vertices:
            coordenadas.append((vertex.x, vertex.y))
            
        # Dibujar borde
        # Borde superior izquierdo del rectángulo
        start_point = (int(coordenadas[0][0]*image_cv.shape[1]), int(coordenadas[0][1]*image_cv.shape[0]))
        # Borde inferior derecho del rectángulo
        end_point = (int(coordenadas[2][0]*image_cv.shape[1]), int(coordenadas[2][1]*image_cv.shape[0]))      
        # Color verde
        color = (0, 255, 0)
        # Ancho de la linea
        thickness = 1
        
        image_cv = cv2.rectangle(image_cv, start_point, end_point, color, thickness)
        
        # Colocar texto
        
        # Texto nombre objeto
        text = object_.name
        i+=1
        # Posición
        position = start_point
        # Tipo
        font_family = cv2.FONT_HERSHEY_SIMPLEX
        # Tamaño
        font_size = 0.37
        # Color verde
        font_color = (0, 255, 0)
        # Stroke
        font_stroke = 1

        image_cv = cv2.putText(image_cv, text, position, font_family, font_size, font_color, font_stroke)
        
        # Se van mostrando las imagenes
        cv2.imshow(window_name, image_cv)
        cv2.waitKey(0) 

In [87]:
detect_objects('test_images\\149048124208.png', 'tc-sc-bi-bigdata-corp-tsod-dev-de643e35c621.json')

test_images\149048124208.png
localized_object_annotations {
  mid: "/m/0n5v01m"
  name: "Bag"
  score: 0.7312207221984863
  bounding_poly {
    normalized_vertices {
      x: 0.11501109600067139
    }
    normalized_vertices {
      x: 0.9331558346748352
    }
    normalized_vertices {
      x: 0.9331558346748352
      y: 0.8522815704345703
    }
    normalized_vertices {
      x: 0.11501109600067139
      y: 0.8522815704345703
    }
  }
}
localized_object_annotations {
  mid: "/m/0hf58v5"
  name: "Luggage & bags"
  score: 0.5019060373306274
  bounding_poly {
    normalized_vertices {
      x: 0.07266440242528915
    }
    normalized_vertices {
      x: 0.9684335589408875
    }
    normalized_vertices {
      x: 0.9684335589408875
      y: 0.8505173325538635
    }
    normalized_vertices {
      x: 0.07266440242528915
      y: 0.8505173325538635
    }
  }
}

Number of objects found: 2
0.7312207221984863

 Figura1
Bag
object: Bag
confidence: 0.7312207221984863
0.5019060969352722

 Figur

In [192]:
# Detección imagen local
def detect_faces(image_path, jason_path):
    """Detects faces in an image."""
    import os
    from google.cloud import vision
    import io
    import cv2
    
    os.environ['GOOGLE_APPLICATION_CREDENTIALS'] = jason_path
    
    # Instanciar cliente
    client = vision.ImageAnnotatorClient()

    with io.open(image_path, 'rb') as image_file:
        content = image_file.read()
        
    print('content: \n')    
    print(content)
    image = vision.Image(content=content)
    response = client.face_detection(image=image)
    faces = response.face_annotations
    i=2
    print('imprimir if: \n')
    if client.face_detection(image=image).error == '':
        print(i)
    
    
    ##############################################
    # Obtener área total de la imagen
    import urllib
    url_open = urllib.request.urlopen('https://firebasestorage.googleapis.com/v0/b/txd-scex-tmgt-geosort-prod.appspot.com/o/delivery%2Flileonb%40intralogistics.cl%2F149045819215.png?alt=media&token=03d6c2db-e887-4140-8dc0-0308f8ee7d36')
    image_cv = np.asarray(bytearray(url_open.read()), dtype="uint8")
    image_cv = cv2.imdecode(image_cv, cv2.IMREAD_COLOR)
    area_total = image_cv.shape[0]*image_cv.shape[1]
    
    import IPython
    url = 'https://firebasestorage.googleapis.com/v0/b/txd-scex-tmgt-geosort-prod.appspot.com/o/delivery%2Flileonb%40intralogistics.cl%2F149045819215.png?alt=media&token=03d6c2db-e887-4140-8dc0-0308f8ee7d36'
    #c = IPython.display.Image(url)
    #print(c)
    
    from PIL import Image
    import requests

    c = Image.open(requests.get(url, stream=True).raw)
    c2 = c.read()
    
    print(c2)

    #with open(path, 'rb') as image_file:
    #    content = image_file.read()
    #image = vision.Image(content=content)
    #b = bytes(image_cv)
    b=url_open.read()
    print('b \n')
    print(b)
    print(image)
    print('r \n')
    r=c2-image
    print(r)
    image2 = vision.Image(content=b)
    response = client.face_detection(image=image2)
    print(client.face_detection(image=image2))
    faces2 = response.face_annotations
    
    ##############################################
    

    # Names of likelihood from google.cloud.vision.enums
    likelihood_name = ('UNKNOWN', 'VERY_UNLIKELY', 'UNLIKELY', 'POSSIBLE',
                       'LIKELY', 'VERY_LIKELY')
    print('Faces:')
    
    image_cv = cv2.imread(image_path)
    window_name = 'Image'
    
    i=1
    for face in faces:
        print('Figura' + str(i))
        print('detection_conidence: {}'.format(face.detection_confidence))
        print('anger: {}'.format(likelihood_name[face.anger_likelihood]))
        print('joy: {}'.format(likelihood_name[face.joy_likelihood]))
        print('surprise: {}'.format(likelihood_name[face.surprise_likelihood]))

        vertices = (['({},{})'.format(vertex.x, vertex.y)
                    for vertex in face.bounding_poly.vertices])
        
        coordenadas = []
        for vertex in face.bounding_poly.vertices:
            coordenadas.append((vertex.x, vertex.y))

        print('face bounds: {}'.format(','.join(vertices)))
        
        # Dibujar borde
        # represents the top left corner of rectangle
        start_point = coordenadas[0]

        # represents the bottom right corner of rectangle
        end_point = coordenadas[2]

        # Blue color in BGR
        color = (0, 255, 0)

        # Line thickness of 2 px
        thickness = 1
        
        image_cv = cv2.rectangle(image_cv, start_point, end_point, color, thickness)
        
        # Colocar texto
        
        #text = str(round((face.detection_confidence),2)) + 'figura ' + str(i)
        text = 'figura ' + str(i)
        i+=1
        print(coordenadas[0][0])
        position = (coordenadas[0][0], coordenadas[0][1]+10)
        font_family = cv2.FONT_HERSHEY_SIMPLEX
        font_size = 0.35
        font_color = (0, 255, 0)
        font_stroke = 1
        
        image_cv = cv2.putText(image_cv, text, position, font_family, font_size, font_color, font_stroke)
        
    cv2.imshow(window_name, image_cv)
    cv2.waitKey(0) 
    
    if response.error.message:
        raise Exception(
            '{}\nFor more info on error messages, check: '
            'https://cloud.google.com/apis/design/errors'.format(
                response.error.message))

In [193]:
detect_faces('C:\\Users\\ext_fantmiraudal\\Desktop\\prueba.png', 'tc-sc-bi-bigdata-corp-tsod-dev-de643e35c621.json')

content: 

b'\x89PNG\r\n\x1a\n\x00\x00\x00\rIHDR\x00\x00\x00\xb0\x00\x00\x00\xea\x08\x02\x00\x00\x00\x8d\xc0X\x98\x00\x00\x00\x01sRGB\x00\xae\xce\x1c\xe9\x00\x00\x00\x03sBIT\x08\x08\x08\xdb\xe1O\xe0\x00\x00 \x00IDATx\x9cl\xbbg\x90]\xd9u\x1e\xba\xd6\xde\'\xdc\x1c:G\xa0\x81\x06\xd0H\x83A\x06&\xe7\xe1\x04\x92C\x0e)Q\xa4D\xcb\x12\x95\xec\xb2\xe5\xa8g\xb9\x9e\xcb\xaf\\\xb6\xdfs\xd5\xb3,K\xa5g\x97\xa2E\xd1"E\x89\x12)q(\x92\xc3\xc9\x83\xc1\x00\x830\x83\xd4\x08\r\xa0s\xbe}\xf3=i\xef\xb5\xd6\xfb\xd1 m\xcb\xdau\xea\xd4\xb9\xb7\xea\xdc{\xea\xaco\xaf\xfd}\xdfZ\x1b\xff\xf0O\xfeT\x94\x02"\x05\xac\\\x85(q\xc2\x0c\xe4:\x0e\x91aA\x10!c=\xad\xfc<1\xe9\xedc\xf7ml\xdc\t\x9a6NB\x04=<4V\xca\xe5\xdbA\xe3\xa1\x07\x9f\xd0 \xeb\xcd\xce\x1f\xff\xd9\x97mh<\x85\xc5\xaeB\xb9\xdc\xd5i\xaf\xb6\x9b\x9dt\xd9\x9f\x9f\xde\x10W\xe7\xb3\xc2\x94P\xd2]k7mb\x88\xe1\xc1\xa3\'\xcf\x9c={\xe2\xe4\xb1\xf7.\x9eE\xa1\xa1\xee>\x13\xd6W[\xcd8\xc0\x04\x1c\x81\xc4\x1a\xe3j1$\x1e\xfa\x81\xa1TZ\x172\xf1\xc2\xba\xb1\xa4\xba\xca\xb9F#\x16c\

imprimir if: 



AttributeError: read

In [51]:
def promedio_tupla(tupla1, tupla2):
    n = len(tupla1)
    prom_tupla = ()
    for indice in range(n):
        prom_tupla += (int((tupla1[indice] + tupla2[indice])/n),)
    return prom_tupla

In [9]:
# Detección imagen remota
def detect_faces_uri(uri, jason_path):
    """Detects faces in the file located in Google Cloud Storage or the web."""
    import os
    from google.cloud import vision
    
    os.environ['GOOGLE_APPLICATION_CREDENTIALS'] = jason_path
    
    client = vision.ImageAnnotatorClient()
    image = vision.Image()
    image.source.image_uri = uri

    response = client.face_detection(image=image)
    faces = response.face_annotations
    if len(faces) == 0:
        print('lista vacia')
        
    print(faces)

    # Names of likelihood from google.cloud.vision.enums
    likelihood_name = ('UNKNOWN', 'VERY_UNLIKELY', 'UNLIKELY', 'POSSIBLE',
                       'LIKELY', 'VERY_LIKELY')
    print('Faces:')

    for face in faces:
        print('anger: {}'.format(likelihood_name[face.anger_likelihood]))
        print('joy: {}'.format(likelihood_name[face.joy_likelihood]))
        print('surprise: {}'.format(likelihood_name[face.surprise_likelihood]))

        vertices = (['({},{})'.format(vertex.x, vertex.y)
                    for vertex in face.bounding_poly.vertices])

        print('face bounds: {}'.format(','.join(vertices)))

    if response.error.message:
        raise Exception(
            '{}\nFor more info on error messages, check: '
            'https://cloud.google.com/apis/design/errors'.format(
                response.error.message))

In [10]:
detect_faces_uri('https://firebasestorage.googleapis.com/v0/b/txd-scex-tmgt-geosort-prod.appspot.com/o/delivery%2Fvsantis%40pipau.cl%2F149046188492.png?alt=media&token=27a2ccda-45d7-4da5-9f43-14db38bc77d7', 'tc-sc-bi-bigdata-corp-tsod-dev-de643e35c621.json')

lista vacia
[]
Faces:


Exception: We can not access the URL currently. Please download the content and pass it in.
For more info on error messages, check: https://cloud.google.com/apis/design/errors

In [ ]:
"auto_ml_jason\\tc-sc-bi-bigdata-corp-tsod-dev-05c522e8964b.json"
'tc-sc-bi-bigdata-corp-tsod-dev-de643e35c621.json'

In [24]:
detect_faces_uri('gs://cloud-samples-data/vision/face/faces.jpeg', 'clave2\\tc-sc-bi-bigdata-corp-tsod-dev-739359c1cf2a.json')

[bounding_poly {
  vertices {
    x: 1077
    y: 157
  }
  vertices {
    x: 2146
    y: 157
  }
  vertices {
    x: 2146
    y: 1399
  }
  vertices {
    x: 1077
    y: 1399
  }
}
fd_bounding_poly {
  vertices {
    x: 1112
    y: 407
  }
  vertices {
    x: 1946
    y: 407
  }
  vertices {
    x: 1946
    y: 1270
  }
  vertices {
    x: 1112
    y: 1270
  }
}
landmarks {
  type_: LEFT_EYE
  position {
    x: 1368.748046875
    y: 739.095703125
    z: 0.002460479736328125
  }
}
landmarks {
  type_: RIGHT_EYE
  position {
    x: 1660.6104736328125
    y: 751.5844116210938
    z: -117.06495666503906
  }
}
landmarks {
  type_: LEFT_OF_LEFT_EYEBROW
  position {
    x: 1284.32080078125
    y: 666.6148681640625
    z: 63.41505813598633
  }
}
landmarks {
  type_: RIGHT_OF_LEFT_EYEBROW
  position {
    x: 1418.9249267578125
    y: 671.494140625
    z: -83.82395935058594
  }
}
landmarks {
  type_: LEFT_OF_RIGHT_EYEBROW
  position {
    x: 1556.9578857421875
    y: 672.2199096679688
    z: -139

In [11]:
os.listdir('C:\\Users\\ext_fantmiraudal\\Desktop\\test_vision_api\\test_images')

['test_image1.jpg', 'test_image2.jpg', 'test_image3.jpg']